In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

model_name='Llama2-7b-hf'
#model_name='Llama-3.2-1B'

### Save per block

In [2]:
qmodel = qlib.QuantizedLlamaForCausalLM.from_pretrained(
	'/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/T256_L16_V2_K2_cbs10_LowBitSym_qtip',
    torch_dtype=torch.float16,
).cpu()

LOL, Im custom!


In [3]:
name = 'T256_L16_V2_K2_cbs10_LowBitSym_qtip'

In [4]:
path_to_save = f'/mnt/ssd_storage/ml/weights/vc_data/{model_name}/per_block_q/{name}'
os.makedirs(path_to_save, exist_ok=True)

lm_head = qmodel.lm_head
torch.save(lm_head, os.path.join(path_to_save, f'lm_head'))

decoder = qmodel.get_decoder()
embed_tokens = decoder.embed_tokens
torch.save(embed_tokens, os.path.join(path_to_save, f'model.embed_tokens'))

rotary_emb=decoder.rotary_emb
torch.save(rotary_emb, os.path.join(path_to_save, f'model.rotary_emb'))

for block_id, block in enumerate(decoder.layers):
	torch.save(block, os.path.join(path_to_save, f'model.layers.{block_id}'))

KeyboardInterrupt: 

### Insert new blocks

In [2]:
qmodel = qlib.QuantizedLlamaForCausalLM.from_pretrained(
	'/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/T256_L16_V2_K2_cbs10_LowBitSym_qtip',
    torch_dtype=torch.float16,
).cpu()

LOL, Im custom!


In [3]:
path_to_new_blocks = '/home/msst/repo/Quantization/logs/llama2-7b/trellis/llama2_trellis_320seq_bs5_ne4/per_block_q_trained'
#path_to_new_blocks = '/home/msst/repo/Quantization/logs/llama2-7b/symquant/llama2_symquant_cb256_vecdim8_C1e4_lossL2/per_block_q_trained'
block_names = sorted(os.listdir(path_to_new_blocks), key=lambda x: int(x.split('.')[-1]) if x.startswith("model.layers") else -1)

for block_name in block_names:
    new_block = torch.load(os.path.join(path_to_new_blocks, block_name), weights_only=False)
    curr_block = qmodel.get_submodule(block_name)

    with torch.no_grad():
        curr_block.load_state_dict(new_block.state_dict())



In [4]:
qmodel.save_pretrained('/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/T256_L16_V2_K2_cbs10_LowBitSym_qtip_ptq_bs5_ne4')